# Principal Component Analysis (PCA)
This file uses latitude and longitude measurements from [Manhattan](http://www.openstreetmap.org/export#map=13/40.7869/-73.9603) to demonstrate how PCA can reduce the dimensionality of a data set. This example is chosen because 2D is easy to visualize, a geographical example is familiar and intuitive, and Manhattan has a particularly regular street grid. 

In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
from glob import glob
from sklearn.decomposition import PCA
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

## Scatter Plot Locations of 311 Call Report
The file "311_Service_Requests_from_2010_to_Present.csv" is exported from [NYC OpenData](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9) for all calls made to 311 from Manhattan on November 9, 2016. A scatter plot ot the locations of all calls shows the shape of the island, which is long and skinny, at an angle compared to the longitude/latitude lines.

In [ ]:
# Define graph limits for each variable to keep them constant and equal range
long_limits = (-74.05, -73.88)
lat_limits = (40.7, 40.87)

In [ ]:
fig, graphs = plt.subplots(1, 2, figsize=(16, 8), sharex=True, sharey=True)

nyc = pd.read_csv(os.path.join("NYCservicecalls","311_Service_Requests_from_2010_to_Present.csv"))
nyc.plot("Longitude", "Latitude", "scatter", color="blue", ax=graphs[0])

graphs[0].set_xlabel("Longitude")
graphs[0].set_ylabel("Latitude")
graphs[0].set_title("Locations of NYC 311 Calls on 11/09/16")
graphs[0].set_xlim(*long_limits)
graphs[0].set_ylim(*lat_limits)

In [ ]:
def readstreetlights(osmfiles):
    longlat = []
    for f in osmfiles:
        etree = ET.parse(f)
        root = etree.getroot()
        children = [child for child in root]
        for c in children:
            if c.getchildren():
                for cc in c.getchildren():
                    if cc.tag == "tag":
                        if cc.attrib['k'] == "highway" and  cc.attrib['v']=="traffic_signals":
                            longlat.append((c.attrib['lon'], c.attrib['lat']))
    return longlat

def plotstreetlights(longlat, ax):
    ax.scatter(*zip(*longlat), color="green", s=2)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title("Locations of Traffic Lights (Street Grid Proxy)")
    return

In [ ]:
files = glob(os.path.join("nycmaps", "map*.osm"))
streetlights = readstreetlights(files)
plotstreetlights(streetlights, graphs[1])
fig

## Show Distribution along the Two Dimensions
Note the distribution of the data points along the two natural dimensions. 

In [ ]:
fig_hist, graphs_hist = plt.subplots(2, 1)
nyc.hist("Longitude", bins=np.arange(*long_limits, 0.01), ax=graphs_hist[0])
graphs_hist[0].set_xlim(long_limits)
nyc.hist("Latitude", bins=np.arange(*lat_limits, 0.01), ax=graphs_hist[1])
graphs_hist[1].set_xlim(lat_limits)
print("Standard deviation of Longitude: ", nyc.Longitude.std())
print("Standard deviation of Latitude: ", nyc.Latitude.std())

PCA is a matrix operation, and after transformation the data is normalized with a vector norm of 1, with distribution around 0. Below we shift and normalize the latitude and longitude data so it is in the same units as the post-transformation histogram.

In [ ]:
# Normalize to compare with output of PCA transformation
fig_var, graphs_var = plt.subplots(2, 1, sharex=True)

temp = nyc.Longitude.dropna()
temp -= temp.mean() 
nyc["lon_norm"] = temp/np.linalg.norm(temp)
temp = nyc.Latitude.dropna()
temp -= temp.mean()
nyc["lat_norm"] = temp/np.linalg.norm(temp)

nyc.hist("lon_norm", bins=np.arange(-0.06, 0.07, 0.01), ax=graphs_var[0])
nyc.hist("lat_norm", bins=np.arange(-0.06, 0.07, 0.01), ax=graphs_var[1])

print(nyc["lon_norm"].std())
print(nyc["lat_norm"].std())

## PCA
Find the principal components of the 311 call dataset and draw the vector(s) on the scatter plots (you can choose to plot just the first component "for v in pca.components\_[:1]" or both "for v in pca.components\_". Note that the first principal component runs almost parallel to the avenues, along the length of the island.

In [ ]:
pca = PCA(n_components=2) # initialize the object
X = nyc[["Longitude", "Latitude"]].dropna().as_matrix() # prepare the data matrix
pca.fit(X) # do the transformation
# Graph the output
for ax in graphs:
    for v in pca.components_[:1]:
        ax.quiver(-74, 40.74, *v, scale=1.5, angles='xy', color="red") # the length of the vector is inverse to the scale
    ax.set_xlim(*long_limits)
    ax.set_ylim(*lat_limits)
fig

Scatter plot the transformed data. Note that the first componenet in the PCA output contains most of the variability (runs along the length of the island).


TODO: finish this note; make PCA graph have same range x & y; write one more note.

In [ ]:
fig_pca, graph_pca = plt.subplots(1, 1)
graph_pca.scatter(pca.fit_transform(X)[:,0], pca.fit_transform(X)[:,1])
graph_pca.set_ylim(*graph_pca.get_xlim()) # make the y range equal to x range to show variance difference
graph_pca.set_xlabel("PCA Component 1")
graph_pca.set_ylabel("PCA Component 2")

In [ ]:
fig_pcahist, graph_pcahist = plt.subplots(1, 1)
plt.hist(pca.fit_transform(X)[:,0], label="PCA Component 1")
plt.hist(pca.fit_transform(X)[:,1], label="PCA Component 2")
graph_pcahist.legend()
print("Standard deviation along PCA Component 1: ", pca.fit_transform(X)[:,0].std())
print("Standard deviation along PCA Component 2: ", pca.fit_transform(X)[:,1].std())
print("PCA explained variance ratio for components [1, 2]: ", pca.explained_variance_ratio_)